# RAG Chatbot With Audio



In [1]:
# Import the required packages 
import os
from langchain_core.prompts import ChatPromptTemplate
from langchain_openai import ChatOpenAI
from langchain_community.document_loaders import UnstructuredHTMLLoader 
from langchain_openai import OpenAIEmbeddings
from langchain_core.runnables import RunnablePassthrough
from langchain_core.output_parsers import StrOutputParser
from langchain_text_splitters import RecursiveCharacterTextSplitter
import openai
from langchain_community.document_loaders import PyPDFLoader
import pathlib
from langchain.vectorstores import Chroma

In [2]:
# Open AI Key 
openai_api_key = os.environ["OPENAI_API_KEY"]

In [17]:
import pathlib
import openai

# Open the audio file for processing 
audio_file_path = pathlib.Path("user_input_audio.mp3")

with audio_file_path.open("rb") as audio_file:
    user_transcript = openai.audio.transcriptions.create(
        model="whisper-1",
        file=audio_file
    )

user_transcript_response = user_transcript.text

print(user_transcript_response)

The procedure EPB system fault warning has appeared. What does this mean and what should I do about it?


### Building the RAG

In [6]:
# Load document 
loader = PyPDFLoader(file_path="car_manual.pdf")
car_docs = loader.load() 

In [7]:
# Define Splitter 
rc_splitter = RecursiveCharacterTextSplitter(
    separators=["\n\n", "\n", " ", ""], 
    chunk_size=1000,
    chunk_overlap=200
) 

docs = rc_splitter.split_documents(car_docs)

In [8]:
# Define embedding function 
embedding_function = OpenAIEmbeddings(api_key=openai_api_key, model='text-embedding-3-small') 

# Store the embeddings in Chroma vector database
vectorstore = Chroma.from_documents(
    documents=docs, 
    embedding=embedding_function, 
)

# Create Chroma retriever
retriever = vectorstore.as_retriever()

In [9]:
# Initialize LLM
llm = ChatOpenAI(model="gpt-4o-mini", api_key=openai_api_key, temperature=0)

In [10]:
# Define Prompt Template
prompt = ChatPromptTemplate.from_template("You are an assistant for question-answering tasks. Use the following pieces of retrieved context to answer the question. If you don't know the answer, just say that you don't know. Use three sentences maximum and keep the answer concise.\nQuestion: {question} \nContext: {context} \nAnswer:")

In [11]:
# Define RAG chain
rag_chain = (
    {"context": retriever, "question": RunnablePassthrough()} 
    | prompt 
    | llm 
)

In [23]:
# Invoke RAG chain with a specific query
query = "The Gasoline Particular Filter Full warning has appeared. What does this mean and what should I do about it?"

llm_response = rag_chain.invoke(query).content

print(llm_response)

The Gasoline Particular Filter Full warning indicates that the gasoline particulate filter is full. You should consult an MG Authorised Repairer as soon as possible to address this issue.


In [25]:
# Text to speech 
chatbot_audio = openai.audio.speech.create(
    model="gpt-4o-mini-tts", 
    voice="onyx", 
    input=llm_response
    )

audio_file = chatbot_audio.stream_to_file("user_output_1.mp3")